- references
    - https://zhuanlan.zhihu.com/p/675273498?utm_id=0
- padding_side:
    - transformers 的 Tokenizer 默认是 padding 右边，

In [2]:
import os
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [19]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

### model & tokenizer

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", 
                                             torch_dtype=torch.float16).to(device)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### pad_token

In [7]:
print(tokenizer.bos_token, tokenizer.bos_token_id)
print(tokenizer.eos_token, tokenizer.eos_token_id)
print(tokenizer.pad_token, tokenizer.pad_token_id)

<s> 1
</s> 2
None None


In [8]:
# 大多数的 LLM 默认都没有 pad_token，如果不设置，会报错
tokenizer.pad_token = tokenizer.eos_token

In [9]:
print(tokenizer.bos_token, tokenizer.bos_token_id)
print(tokenizer.eos_token, tokenizer.eos_token_id)
print(tokenizer.pad_token, tokenizer.pad_token_id)

<s> 1
</s> 2
</s> 2


In [10]:
inputs = tokenizer(
    ["hello llama", "who are you?"], padding=True, return_tensors="pt"
)
inputs = {k: v.to(device) for k, v in inputs.items()}
inputs

{'input_ids': tensor([[    1, 22172, 11148,  3304,     2],
         [    1,  1058,   526,   366, 29973]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 0],
         [1, 1, 1, 1, 1]], device='cuda:0')}

In [22]:
GenerationConfig.from_pretrained('meta-llama/Llama-2-7b-hf')

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9
}

In [11]:
generated_ids = model.generate(**inputs, )

In [15]:
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[1])

who are you? (part 2)
What does it mean to be a human? How do we find out who we are? What does it mean to be a human?
I’m not a very good person, but I’m not a bad person either. I’m just a person.
I’m not a good person, but I’m not a bad person either. I’m just a person.
I’m not a bad person, but I’m not a good person either. I’m just a person.
I’m not a person, but I’m not a bad person either. I’m just a person.
I’m not a good person, but I’m not a bad person either. I’m just a person.
I’m not a bad person, but I’m not a good person either. I’m just a person.
I’m not a person, but I’m not a bad person either. I’m just a person.
I’m not a good person, but I’m not a bad person either. I’m just a person.
I’m not a bad person, but I’m not a good person either. I’m just a person.
I’m not a person, but I’m not a bad person either. I’m just a person.
I’m not a good person, but I’m not a bad person either. I’m just a person.
I’m not a bad person, but I’m not a good person either. I’m just a

### padding_side